# YOLO Train

## Read data

In [5]:
import os
import numpy as np
from data_gen import data_generator, shuffle_split
from utils import read_anchors

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
xml_path = 'E:/data/The Oxford-IIIT Pet Dataset/annotations/xmls'
image_path = 'E:/data/The Oxford-IIIT Pet Dataset/images/'
file_list = os.listdir(xml_path)
num_classes = 2
classes_name = {'cat': 0, 'dog': 1}

### Read anchors

In [3]:
anchors = read_anchors('model/pet_anchors.txt')
num_anchors = len(anchors) // 3

### Generate data

In [22]:
# shuffle split
cat_list = []
dog_list = []
for file in file_list:
    filename = file.split('.')[0]
    if file[0].isupper():
        cat_list.append(filename)
    else:
        dog_list.append(filename)
del file_list
data_set = [cat_list, dog_list]
train_list = []
test_list = []
for file_list in data_set:
    tmp_train_list, tmp_test_list = shuffle_split(file_list)
    train_list += tmp_train_list
    test_list += tmp_test_list

In [24]:
batch_size = 32
train_steps = len(train_list)//batch_size
test_steps = len(test_list)//batch_size
train_gen = data_generator(train_list, image_path, xml_path, classes_name, anchors, batch_size=batch_size)
test_gen = data_generator(test_list, image_path, xml_path, classes_name, anchors, batch_size=batch_size, loop=False)

In [25]:
print(train_steps, test_steps)
print(len(train_list), len(test_list))

103 11
3319 367


## Build new model

In [27]:
from yolo_model import yolo_model, yolo_loss
from keras.layers import Input, Lambda
from keras.models import Model, load_model
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
def build_model():
    inputs = Input((416, 416, 3), dtype='float32')
    y1, y2, y3 = yolo_model(inputs, num_anchors, num_classes)
    model = Model(inputs, [y1, y2, y3])
    model.load_weights("model/yolo_base.h5", by_name=True)

    for layer in model.layers[:-3]:
        layer.trainable = False
        
    y_tensor = [Input(shape=(13, 13, num_anchors, num_classes+5), name='y_input_1'),
                Input(shape=(26, 26, num_anchors, num_classes+5), name='y_input_2'),
                Input(shape=(52, 52, num_anchors, num_classes+5), name='y_input_3')]
    loss_layer = Lambda(yolo_loss, name='yolo_loss', 
                        arguments={'anchors': anchors, 'num_classes': num_classes})([*model.output, *y_tensor])

    train_model = Model([model.input, *y_tensor], loss_layer)
    
    return train_model

In [11]:
K.clear_session()
train_model = build_model()
train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 416, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 416, 416, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

batch_normalization_11 (BatchNo (None, 52, 52, 128)  512         conv2d_11[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_11 (LeakyReLU)      (None, 52, 52, 128)  0           batch_normalization_11[0][0]     
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 52, 52, 256)  294912      leaky_re_lu_11[0][0]             
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 52, 52, 256)  1024        conv2d_12[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_12 (LeakyReLU)      (None, 52, 52, 256)  0           batch_normalization_12[0][0]     
__________________________________________________________________________________________________
add_4 (Add

__________________________________________________________________________________________________
leaky_re_lu_22 (LeakyReLU)      (None, 52, 52, 256)  0           batch_normalization_22[0][0]     
__________________________________________________________________________________________________
add_9 (Add)                     (None, 52, 52, 256)  0           add_8[0][0]                      
                                                                 leaky_re_lu_22[0][0]             
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 52, 52, 128)  32768       add_9[0][0]                      
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 52, 52, 128)  512         conv2d_23[0][0]                  
__________________________________________________________________________________________________
leaky_re_l

batch_normalization_33 (BatchNo (None, 26, 26, 512)  2048        conv2d_33[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_33 (LeakyReLU)      (None, 26, 26, 512)  0           batch_normalization_33[0][0]     
__________________________________________________________________________________________________
add_14 (Add)                    (None, 26, 26, 512)  0           add_13[0][0]                     
                                                                 leaky_re_lu_33[0][0]             
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 26, 26, 256)  131072      add_14[0][0]                     
__________________________________________________________________________________________________
batch_normalization_34 (BatchNo (None, 26, 26, 256)  1024        conv2d_34[0][0]                  
__________

zero_padding2d_5 (ZeroPadding2D (None, 27, 27, 512)  0           add_19[0][0]                     
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 13, 13, 1024) 4718592     zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 13, 13, 1024) 4096        conv2d_44[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_44 (LeakyReLU)      (None, 13, 13, 1024) 0           batch_normalization_44[0][0]     
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 13, 13, 512)  524288      leaky_re_lu_44[0][0]             
__________________________________________________________________________________________________
batch_norm

batch_normalization_55 (BatchNo (None, 13, 13, 512)  2048        conv2d_55[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_55 (LeakyReLU)      (None, 13, 13, 512)  0           batch_normalization_55[0][0]     
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 13, 13, 1024) 4718592     leaky_re_lu_55[0][0]             
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 13, 13, 1024) 4096        conv2d_56[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_56 (LeakyReLU)      (None, 13, 13, 1024) 0           batch_normalization_56[0][0]     
__________________________________________________________________________________________________
conv2d_57 

batch_normalization_69 (BatchNo (None, 52, 52, 128)  512         conv2d_71[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_69 (LeakyReLU)      (None, 52, 52, 128)  0           batch_normalization_69[0][0]     
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 52, 52, 256)  294912      leaky_re_lu_69[0][0]             
__________________________________________________________________________________________________
batch_normalization_70 (BatchNo (None, 52, 52, 256)  1024        conv2d_72[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_70 (LeakyReLU)      (None, 52, 52, 256)  0           batch_normalization_70[0][0]     
__________________________________________________________________________________________________
conv2d_73 

## Train

In [12]:
train_model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min'),
             ModelCheckpoint(filepath='model/yolo_train.h5', monitor='val_loss', verbose=0, save_best_only=True)
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)]
train_model.fit_generator(train_gen, train_steps, epochs=50, validation_data=test_gen, validation_steps=test_steps, callbacks=callbacks)

In [16]:
train_model.save('model/yolo_train.h5')

### Test loss

In [10]:
data_index = 0
input_tensor = np.expand_dims(X[data_index], 0)
y_tensor = [np.expand_dims(y_true[0][data_index], 0), np.expand_dims(y_true[1][data_index], 0), np.expand_dims(y_true[2][data_index], 0)]


In [17]:
loss = train_model.output
sess = K.get_session()
loss_info = sess.run(loss, feed_dict={train_model.input[0]: input_tensor, 
                                      train_model.input[1]: y_tensor[0], 
                                      train_model.input[2]: y_tensor[1], 
                                      train_model.input[3]: y_tensor[2]})

In [18]:
print(loss_info)

[3.7198381  2.7847905  0.47783184 2.864696   3.0248    ]
